In [5]:
import numpy as np
import pandas as pd
from scipy import sparse
from matplotlib import pyplot as plt
import seaborn
%matplotlib inline

### Data reading

In [86]:
scrobs = pd.read_csv("./data/lastfm_user_scrobbles.csv")

In [87]:
scrobs

,user_id,artist_id,scrobbles
0,1,4562,13883
1,1,10191,11690
2,1,494,11351
3,1,6673,10300
4,1,8402,8983
...,...,...,...
92787,1892,10098,278
92788,1892,8660,263
92789,1892,3274,258
92790,1892,4240,232


### Make scipy csr sparse matrix from lil with users as rows and artist as column with # scrobbles as value

In [36]:
scrobsLil = sparse.lil_matrix((max(scrobs["user_id"])+1, max(scrobs["artist_id"])+1)) # 0 indexing
scrobsLil[scrobs["user_id"], scrobs["artist_id"]] = scrobs["scrobbles"]
scrobs = scrobsLil.tocsr()

### Change #listens to ratings

This is to make collaborative filtering work, rather than values like 100, 90, 80 for the top 3 artists of a user, these would be scores representing how they might rate those artists. 

1. Divide by max
    * result: 1, .9, .8
    * ranking of a user's artists
    * 
2. Divide by sum
    * result: 0.37, 0.33, 0.29
    * proportion of total listens that artist occupies
    * users with many artists have lower ratings
    * ranking doesn't affect rating, so 

In [84]:
denom_max = np.repeat(scrobs.max(axis=1).A, scrobs.getnnz(axis=1))
scrobs.data /= denom_max


array([0.09472016, 0.81761867, 0.18346179, ..., 0.22556762, 0.26406713,
       0.65794669])

### 

In [80]:
# edges=[[1,2],[3,4],[1,5],[5,3]]
toy = np.array([[1,1,2,3,3], [2,3,1,1,2]])
toydata = np.array([100,10,200,300,30])
lil = sparse.lil_matrix((3,3))
lil[toy[0,:]-1,toy[1,:]-1] = toydata
csr = lil.tocsr()
csr.getnnz(axis=1)
csr.max(axis=1).A
denom = np.repeat(csr.max(axis=1).A,csr.getnnz(axis=1))
csr.data = csr.data/denom
csr.A

array([[0. , 1. , 0.1],
       [1. , 0. , 0. ],
       [1. , 0.1, 0. ]])